In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import requests
import time
import json
import string
from datetime import date 
from scipy.stats import linregress

#Notes: https://data.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Behavioral/hn4x-zwk7



In [18]:
#Import the API Key
#from api_key import api_key
#Don't need API Key

#import data
#url = "https://data.cdc.gov/resource/hn4x-zwk7.json"
data_df = pd.read_csv(r"Resources\Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System_20231101.csv")

#response = requests.get(url)
#data = response.json()

#data_df.info()
data_df

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
0,2020,2020,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,NaN,PA,PA1,Q047,VALUE,59,Race/Ethnicity,Hispanic,RACE,RACEHIS
1,2014,2014,GU,Guam,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(13.444304, 144.793731)",OWS,OWS1,Q036,VALUE,66,Education,High school graduate,EDU,EDUHSGRAD
2,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q036,VALUE,59,Income,"$50,000 - $74,999",INC,INC5075
3,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q037,VALUE,59,Income,Data not reported,INC,INCNR
4,2015,2015,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 300 min...,NaN,Value,...,NaN,PA,PA1,Q045,VALUE,59,Income,"Less than $15,000",INC,INCLESS15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88624,2021,2021,ND,North Dakota,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming vegetab...,NaN,Value,...,"(47.47531977900047, -100.11842104899966)",FV,FV1,Q019,VALUE,38,Gender,Male,GEN,MALE
88625,2021,2021,PR,Puerto Rico,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(18.220833, -66.590149)",PA,PA1,Q047,VALUE,72,Income,"$35,000 - $49,999",INC,INC3550
88626,2021,2021,WI,Wisconsin,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(44.39319117400049, -89.81637074199966)",PA,PA1,Q047,VALUE,55,Income,"$75,000 or greater",INC,INC75PLUS
88627,2021,2021,UT,Utah,Behavioral Risk Factor Surveillance System,Fruits and Vegetables,Fruits and Vegetables - Behavior,Percent of adults who report consuming fruit l...,NaN,Value,...,"(39.360700171000474, -111.58713063499971)",FV,FV1,Q018,VALUE,49,Income,"$50,000 - $74,999",INC,INC5075


In [21]:

list_of_categories = data_df['StratificationCategory1'].unique()
list_of_categories


array(['Race/Ethnicity', 'Education', 'Income', 'Age (years)', 'Gender',
       'Total', nan], dtype=object)

In [22]:
ethnicity_df = [data_df['StratificationCategory1'] == 'Race/Ethnicity']
ethnicity_df

[0         True
 1        False
 2        False
 3        False
 4        False
          ...  
 88624    False
 88625    False
 88626    False
 88627    False
 88628    False
 Name: StratificationCategory1, Length: 88629, dtype: bool]

In [23]:
education_df = data_df[data_df['StratificationCategory1'] == 'Education']
education_df

,YearEnd,LocationAbbr,Question,Data_Value,Sample_Size,Race/Ethnicity,Education,Gender,StratificationCategory1,Stratification1
1,2014,GU,Percent of adults aged 18 years and older who ...,29.3,842.0,NaN,High school graduate,NaN,Education,High school graduate
7,2012,DC,Percent of adults aged 18 years and older who ...,31.6,243.0,NaN,Less than high school,NaN,Education,Less than high school
10,2015,GU,Percent of adults who engage in no leisure-tim...,30.5,559.0,NaN,High school graduate,NaN,Education,High school graduate
21,2012,US,Percent of adults aged 18 years and older who ...,35.1,120032.0,NaN,Some college or technical school,NaN,Education,Some college or technical school
43,2016,RI,Percent of adults aged 18 years and older who ...,20.1,2071.0,NaN,College graduate,NaN,Education,College graduate
...,...,...,...,...,...,...,...,...,...,...
88571,2021,DC,Percent of adults who report consuming fruit l...,45.1,101.0,NaN,Less than high school,NaN,Education,Less than high school
88580,2021,UT,Percent of adults who engage in no leisure-tim...,9.3,4279.0,NaN,College graduate,NaN,Education,College graduate
88581,2021,GU,Percent of adults aged 18 years and older who ...,26.1,547.0,NaN,High school graduate,NaN,Education,High school graduate
88615,2021,VT,Percent of adults aged 18 years and older who ...,32.6,1372.0,NaN,Some college or technical school,NaN,Education,Some college or technical school


In [24]:
income_df = data_df[data_df['StratificationCategory1'] == 'Income']
income_df

,YearEnd,LocationAbbr,Question,Data_Value,Sample_Size,Race/Ethnicity,Education,Gender,StratificationCategory1,Stratification1
2,2013,US,Percent of adults aged 18 years and older who ...,28.8,62562.0,NaN,NaN,NaN,Income,"$50,000 - $74,999"
3,2013,US,Percent of adults aged 18 years and older who ...,32.7,60069.0,NaN,NaN,NaN,Income,Data not reported
4,2015,US,Percent of adults who achieve at least 300 min...,26.6,30904.0,NaN,NaN,NaN,Income,"Less than $15,000"
8,2015,PR,Percent of adults who engage in no leisure-tim...,38.1,421.0,NaN,NaN,NaN,Income,"$25,000 - $34,999"
13,2012,WY,Percent of adults who engage in no leisure-tim...,32.3,484.0,NaN,NaN,NaN,Income,"Less than $15,000"
...,...,...,...,...,...,...,...,...,...,...
88620,2021,WY,Percent of adults who report consuming vegetab...,28.2,295.0,NaN,NaN,NaN,Income,"$15,000 - $24,999"
88625,2021,PR,Percent of adults who engage in no leisure-tim...,37.4,347.0,NaN,NaN,NaN,Income,"$35,000 - $49,999"
88626,2021,WI,Percent of adults who engage in no leisure-tim...,11.8,850.0,NaN,NaN,NaN,Income,"$75,000 or greater"
88627,2021,UT,Percent of adults who report consuming fruit l...,37.1,1451.0,NaN,NaN,NaN,Income,"$50,000 - $74,999"


In [25]:
age_df = data_df[data_df['StratificationCategory1'] == 'Age (years)']
age_df

,YearEnd,LocationAbbr,Question,Data_Value,Sample_Size,Race/Ethnicity,Education,Gender,StratificationCategory1,Stratification1
9,2011,AL,Percent of adults aged 18 years and older who ...,35.2,598.0,NaN,NaN,NaN,Age (years),25 - 34
17,2015,GU,Percent of adults aged 18 years and older who ...,33.7,285.0,NaN,NaN,NaN,Age (years),55 - 64
24,2015,PR,Percent of adults aged 18 years and older who ...,42.0,997.0,NaN,NaN,NaN,Age (years),55 - 64
25,2011,US,Percent of adults who engage in no leisure-tim...,16.9,20923.0,NaN,NaN,NaN,Age (years),18 - 24
29,2015,PR,Percent of adults who engage in no leisure-tim...,50.4,862.0,NaN,NaN,NaN,Age (years),45 - 54
...,...,...,...,...,...,...,...,...,...,...
88609,2021,US,Percent of adults aged 18 years and older who ...,35.9,57677.0,NaN,NaN,NaN,Age (years),45 - 54
88610,2021,LA,Percent of adults aged 18 years and older who ...,31.7,934.0,NaN,NaN,NaN,Age (years),55 - 64
88617,2021,SC,Percent of adults who report consuming fruit l...,41.0,854.0,NaN,NaN,NaN,Age (years),35 - 44
88621,2021,HI,Percent of adults who report consuming fruit l...,56.5,467.0,NaN,NaN,NaN,Age (years),18 - 24


In [26]:
gender_df = data_df[data_df['StratificationCategory1'] == 'Gender']
gender_df


,YearEnd,LocationAbbr,Question,Data_Value,Sample_Size,Race/Ethnicity,Education,Gender,StratificationCategory1,Stratification1
23,2013,US,Percent of adults who engage in no leisure-tim...,27.9,266452.0,NaN,NaN,Female,Gender,Female
32,2020,ME,Percent of adults who engage in no leisure-tim...,21.7,4518.0,NaN,NaN,Male,Gender,Male
36,2016,WY,Percent of adults aged 18 years and older who ...,29.8,2293.0,NaN,NaN,Female,Gender,Female
41,2016,NY,Percent of adults aged 18 years and older who ...,25.9,14316.0,NaN,NaN,Male,Gender,Male
49,2016,AR,Percent of adults aged 18 years and older who ...,38.0,1918.0,NaN,NaN,Male,Gender,Male
...,...,...,...,...,...,...,...,...,...,...
88568,2021,CO,Percent of adults who engage in no leisure-tim...,19.0,5118.0,NaN,NaN,Female,Gender,Female
88579,2021,MT,Percent of adults aged 18 years and older who ...,31.7,3046.0,NaN,NaN,Male,Gender,Male
88597,2021,DE,Percent of adults aged 18 years and older who ...,29.7,1709.0,NaN,NaN,Female,Gender,Female
88607,2021,NH,Percent of adults who engage in no leisure-tim...,21.5,3638.0,NaN,NaN,Female,Gender,Female


In [ ]:
totals_df = data_df[data_df['StratificationCategory1'] == 'Total']
totals_df

In [5]:
response.content



b'[{"yearstart":"2021","yearend":"2021","locationabbr":"AL","locationdesc":"Alabama","datasource":"Behavioral Risk Factor Surveillance System","class":"Obesity / Weight Status","topic":"Obesity / Weight Status","question":"Percent of adults aged 18 years and older who have an overweight classification","data_value_type":"Value","data_value":"28.5","data_value_alt":"28.5","low_confidence_limit":"21.6","high_confidence_limit":"36.5","sample_size":"232.0","age_years":"18 - 24","geolocation":{"latitude":"32.84057112200048","longitude":"-86.63186076199969","human_address":"{\\"address\\": \\"\\", \\"city\\": \\"\\", \\"state\\": \\"\\", \\"zip\\": \\"\\"}"},"classid":"OWS","topicid":"OWS1","questionid":"Q037","datavaluetypeid":"VALUE","locationid":"01","stratificationcategory1":"Age (years)","stratification1":"18 - 24","stratificationcategoryid1":"AGEYR","stratificationid1":"AGEYR1824"}\n,{"yearstart":"2021","yearend":"2021","locationabbr":"AL","locationdesc":"Alabama","datasource":"Behavio

In [19]:
# Convert data into a Pandas DataFrame
#data_df = pd.DataFrame(data)

#data_df = data_df[['yearend','locationabbr','question','data_value','sample_size','age_years','StratificationCategory1','Stratification1']]
data_df = data_df[['YearEnd','LocationAbbr','Question','Data_Value','Sample_Size','Race/Ethnicity','Education','Gender','StratificationCategory1','Stratification1']]
#data_df['data_value'] = data_df.data_value.astype(float)
#data_df['sample_size'] = data_df.sample_size.astype(float)

#data_df = data_df[data_df['sample_size'].notna()]

data_df.info()
# Show Record Count

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88629 entries, 0 to 88628
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   YearEnd                  88629 non-null  int64  
 1   LocationAbbr             88629 non-null  object 
 2   Question                 88629 non-null  object 
 3   Data_Value               79851 non-null  float64
 4   Sample_Size              79851 non-null  float64
 5   Race/Ethnicity           25320 non-null  object 
 6   Education                12660 non-null  object 
 7   Gender                   6330 non-null   object 
 8   StratificationCategory1  88620 non-null  object 
 9   Stratification1          88620 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 6.8+ MB


In [16]:
 data_df
    

,YearEnd,LocationAbbr,Question,Data_Value,Sample_Size,Race/Ethnicity,Education,Gender,StratificationCategory1,Stratification1
0,2020,US,Percent of adults who engage in no leisure-tim...,30.6,31255.0,Hispanic,NaN,NaN,Race/Ethnicity,Hispanic
1,2014,GU,Percent of adults aged 18 years and older who ...,29.3,842.0,NaN,High school graduate,NaN,Education,High school graduate
2,2013,US,Percent of adults aged 18 years and older who ...,28.8,62562.0,NaN,NaN,NaN,Income,"$50,000 - $74,999"
3,2013,US,Percent of adults aged 18 years and older who ...,32.7,60069.0,NaN,NaN,NaN,Income,Data not reported
4,2015,US,Percent of adults who achieve at least 300 min...,26.6,30904.0,NaN,NaN,NaN,Income,"Less than $15,000"
...,...,...,...,...,...,...,...,...,...,...
88624,2021,ND,Percent of adults who report consuming vegetab...,27.2,2690.0,NaN,NaN,Male,Gender,Male
88625,2021,PR,Percent of adults who engage in no leisure-tim...,37.4,347.0,NaN,NaN,NaN,Income,"$35,000 - $49,999"
88626,2021,WI,Percent of adults who engage in no leisure-tim...,11.8,850.0,NaN,NaN,NaN,Income,"$75,000 or greater"
88627,2021,UT,Percent of adults who report consuming fruit l...,37.1,1451.0,NaN,NaN,NaN,Income,"$50,000 - $74,999"


In [8]:
#Add calculated number of obese people
data_df['calculated_num_obese'] = round(data_df['sample_size'] * data_df['data_value'] * .01)
data_df.head()

,yearend,locationabbr,question,data_value,sample_size,age_years,calculated_num_obese
0,2021,AL,Percent of adults aged 18 years and older who ...,28.5,232.0,18 - 24,66.0
1,2021,AL,Percent of adults who engage in no leisure-tim...,19.3,254.0,18 - 24,49.0
2,2021,AL,Percent of adults who report consuming fruit l...,44.8,223.0,18 - 24,100.0
3,2021,AL,Percent of adults who report consuming vegetab...,24.7,219.0,18 - 24,54.0
4,2021,AL,Percent of adults aged 18 years and older who ...,25.4,232.0,18 - 24,59.0


In [9]:
num_of_participants = data_df['sample_size'].sum()
num_of_participants

1400392.0

In [10]:
#These are the questions this dataset answers
list_of_questions = data_df['question'].unique()
print(sorted(list_of_questions))

['Percent of adults aged 18 years and older who have an overweight classification', 'Percent of adults aged 18 years and older who have obesity', 'Percent of adults who engage in no leisure-time physical activity', 'Percent of adults who report consuming fruit less than one time daily', 'Percent of adults who report consuming vegetables less than one time daily']


In [11]:
#These are the age groups
age_groups = data_df['age_years'].unique()
print(age_groups)

['18 - 24' '25 - 34' '35 - 44' '45 - 54' '55 - 64' '65 or older' nan]


In [12]:
overweight_df = data_df[data_df['question'] == 'Percent of adults aged 18 years and older who have an overweight classification']
overweight_df.head()

,yearend,locationabbr,question,data_value,sample_size,age_years,calculated_num_obese
0,2021,AL,Percent of adults aged 18 years and older who ...,28.5,232.0,18 - 24,66.0
7,2021,AL,Percent of adults aged 18 years and older who ...,21.1,433.0,25 - 34,91.0
12,2021,AL,Percent of adults aged 18 years and older who ...,31.8,521.0,35 - 44,166.0
18,2021,AL,Percent of adults aged 18 years and older who ...,33.8,594.0,45 - 54,201.0
22,2021,AL,Percent of adults aged 18 years and older who ...,32.7,826.0,55 - 64,270.0


In [13]:
obese_df = data_df[data_df['question'] == 'Percent of adults aged 18 years and older who have obesity']
obese_df.head()

,yearend,locationabbr,question,data_value,sample_size,age_years,calculated_num_obese
4,2021,AL,Percent of adults aged 18 years and older who ...,25.4,232.0,18 - 24,59.0
9,2021,AL,Percent of adults aged 18 years and older who ...,46.6,433.0,25 - 34,202.0
13,2021,AL,Percent of adults aged 18 years and older who ...,45.8,521.0,35 - 44,239.0
16,2021,AL,Percent of adults aged 18 years and older who ...,46.4,594.0,45 - 54,276.0
20,2021,AL,Percent of adults aged 18 years and older who ...,44.4,826.0,55 - 64,367.0


In [29]:
no_leisure_df = data_df[data_df['question'] == 'Percent of adults who engage in no leisure-time physical activity']
no_leisure_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 1 to 997
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   yearend               186 non-null    object 
 1   locationabbr          186 non-null    object 
 2   question              186 non-null    object 
 3   data_value            186 non-null    float64
 4   sample_size           186 non-null    float64
 5   age_years             46 non-null     object 
 6   calculated_num_obese  186 non-null    float64
dtypes: float64(3), object(4)
memory usage: 11.6+ KB


In [28]:
no_fruit_df = data_df[data_df['question'] == 'Percent of adults who report consuming fruit less than one time daily']
no_fruit_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 2 to 998
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   yearend               186 non-null    object 
 1   locationabbr          186 non-null    object 
 2   question              186 non-null    object 
 3   data_value            186 non-null    float64
 4   sample_size           186 non-null    float64
 5   age_years             46 non-null     object 
 6   calculated_num_obese  186 non-null    float64
dtypes: float64(3), object(4)
memory usage: 11.6+ KB


In [27]:
no_vegetables_df = data_df[data_df['question'] == 'Percent of adults who report consuming vegetables less than one time daily']
no_vegetables_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 3 to 995
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   yearend               186 non-null    object 
 1   locationabbr          186 non-null    object 
 2   question              186 non-null    object 
 3   data_value            186 non-null    float64
 4   sample_size           186 non-null    float64
 5   age_years             46 non-null     object 
 6   calculated_num_obese  186 non-null    float64
dtypes: float64(3), object(4)
memory usage: 11.6+ KB


In [17]:
alabama_df = data_df[data_df['locationabbr'] == 'AL']
alabama_df.head()

,yearend,locationabbr,question,data_value,sample_size,age_years,calculated_num_obese
0,2021,AL,Percent of adults aged 18 years and older who ...,28.5,232.0,18 - 24,66.0
1,2021,AL,Percent of adults who engage in no leisure-tim...,19.3,254.0,18 - 24,49.0
2,2021,AL,Percent of adults who report consuming fruit l...,44.8,223.0,18 - 24,100.0
3,2021,AL,Percent of adults who report consuming vegetab...,24.7,219.0,18 - 24,54.0
4,2021,AL,Percent of adults aged 18 years and older who ...,25.4,232.0,18 - 24,59.0
